<div style="background-color:#f0f8ff; border-radius:8px; padding:12px; text-align:center;">

# **PRACTICA 2: Árboles y Ensembles**

</div>

*Aprendizaje Automático*

---

**Grupo:** G-7312  
**Número de pareja:** 01  
**Miembros:**  
- Leire Bernárdez Vázquez  
- Carmen Reiné Rueda

---

### **Importaciones**

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_covtype
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel


### **1. Clasificación mediante  ́arboles de clasificación**

---
**1. Cargue los datos del problema**

In [2]:
# Load the Covertype dataset
cov_type = fetch_covtype()
# Separate the features and the target variable
X = cov_type.data
y = cov_type.target

**2. Separe los datos en training (50 %) y test (50 %)**

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42, stratify=y
)

No hace falta escalarlo, lo dice la teacher (los árboles se tragan todo lo que les des)

**3. Utilice la clase DecisionTreeClassifier de Sklearn para resolver el problema
y dé un resultado de test usando la m ́etrica que considere oportuna**

In [4]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
porcentaje = accuracy_score(y_test, y_pred)
print("Accuracy:", porcentaje*100, "%")

Accuracy: 92.37055344812156 %


**4. ¿Había missing values en nuestro problema? ¿Qué efecto tendría en los árboles no completar los missing values ?**

In [5]:
np.isnan(X_train).sum()==0

True

Si existieran y no se imputaran, DecisionTreeClassifier daría error al entrenar, porque los árboles no soportan missing values; requieren que todos los datos estén completos para funcionar. Para solucionar este problema, se podrían imputar con media, mediana o un valor concreto.

**5. ¿Con qué parémetro podría controlarse la profundidad del ́arbol? ¿Qué ocurre si un ́arbol se inicializa sin dar un valor concreto a ese parámetro? ¿Crees que sería una buena práctica no darle ningún valor?**

a) La profundidad máxima del árbol se controla con el parámetro max_depth directamente, que por defecto es max_depth=None. Sin embargo, también se puede controlar indirectamente con min_samples_split, que no divide un nodo si tiene menos muestras que las requeridas, y con min_samples_leaf, que obliga a que cada hoja tenga un mínimo de muestras para evitar que existan ramas muy profundas con pocas muestras. Por lo tanto, con estos valores se reduce la profundidad máxima alcanzada.

b) Crecería tanto como datos se le pasaran sin ningún tipo de límite.

c) No, debido a que sin un límite establecido, los árboles grandes consumen más memoria, tardan más en entrenar y tienden a sobreajustarse más fácilmente. Con lo cual, sería importante definir un max_depth razonable teniendo en cuenta los datos.

**6. ¿Son sensibles los ́arboles al desequilibrio entre clases?**

Sí, los árboles de decisión se ven afectados cuando las clases están desbalanceadas. Tratan de que cada hoja tenga solo un tipo de clase, así que cuando una clase es mayoritaria, las divisiones tienden a favorecerla. Esto puede hacer que las clases minoritarias queden mezcladas o se clasifiquen mal.

Si no se corrige este desequilibrio, el árbol aprenderá principalmente la clase mayoritaria y puede fallar al predecir las clases minoritarias, aunque acierte mucho con la clase que domina.


**7. ¿Existe desequilibrio entre las clases de nuestro problema? ¿Con qué parámetro de la clase DecisionTreeClassifier podrías corregirlo?**

In [6]:
counts = np.bincount(y)  # y debe ser enteros >=0
print({i: count for i, count in enumerate(counts) if count > 0})

{1: 211840, 2: 283301, 3: 35754, 4: 2747, 5: 9493, 6: 17367, 7: 20510}


a) Para comprobar si existe desequilibrio entre las clases del dataset Covtype, hemos utilizado la función np.bincount sobre las etiquetas de cada muestra.  

   Esta función nos permite contar cuántas muestras hay de cada clase y así analizar la distribución de los datos. Los resultados muestran que algunas clases tienen muchas más muestras que otras, confirmando que sí existe desequilibrio entre las clases.  

b) Podríamos corregirlo utilizando el parámetro class_weight='balanced  en DecisionTreeClassifier, que ajusta automáticamente los pesos de cada clase en función de su frecuencia en el dataset.

**8. ¿Cuál es la diferencia entre los parámetros min samples split y min samples leaf ?**

 **`min_samples_split`** 

**Definición según la documentación:** El número mínimo de muestras necesarias para dividir un nodo interno.

Este parámetro decide si un nodo se puede dividir. 
- Si el grupo de datos tiene suficientes muestras según el valor indicado, el nodo se divide en dos o más partes.  
- Si no tiene suficientes, el nodo se queda igual y se convierte en hoja.


**`min_samples_leaf`**  

**Definición según la documentación:** El número mínimo de muestras necesarias para estar en un nodo hoja. Un punto de división a cualquier profundidad solo se considerará si sale a menos muestras de entrenamiento en cada una de las muestras de entrenamiento izquierdo y ramas derechas. Esto puede tener el efecto de suavizar el modelo

Este parámetro se utiliza después de dividir un nodo para comprobar si cada hoja tiene suficientes muestras.  
- Si alguna hoja tiene menos muestras que el valor indicado, la división no se realiza.


**9. ¿Para qué sirve el parémetro criterion, qué valores puede tomar y qué significa o representa cada uno de esos valores?**

El parámetro criterion sirve para decidir qué tan buena es una división en un árbol de decisión. Es decir, indica cómo medir la calidad de los splits.

Los valores que puede tomar son:

- **gini** → utiliza el índice de Gini, que mide la impureza de un nodo.  
  Busca que cada hoja tenga muestras de la misma clase lo más posible.  

- **entropy** → utiliza la ganancia de información de Shannon.  
  Calcula cuánto disminuye la incertidumbre o desorden al hacer un split.  
  
- **log_loss** → también mide la ganancia de información, pero usando pérdida logarítmica.

**10. El ́arbol entrenado, ¿realiza splits utilizando todas las variables disponibles?**

In [7]:
importances = decision_tree.feature_importances_
used_features = [(i, imp) for i, imp in enumerate(importances) if imp > 0]
print("Variables usadas con importancia:", used_features)


Variables usadas con importancia: [(0, 0.3388348139715392), (1, 0.028128228830494227), (2, 0.01730986421399671), (3, 0.06030282813122783), (4, 0.04691144085841901), (5, 0.14839823625394416), (6, 0.031282488311127364), (7, 0.033597830500244244), (8, 0.024419771987702663), (9, 0.14475755736863216), (10, 0.009002493642824017), (11, 0.00303063055395945), (12, 0.013365413460372863), (13, 0.0003924558921933753), (14, 0.00033347464544915387), (15, 0.00982474670015767), (16, 0.0013719987481363033), (17, 0.011252591807065403), (18, 0.00026533767294261915), (19, 0.0005772205308382756), (22, 0.00011716541185293036), (23, 0.0025920958831334253), (24, 0.0018960413093079178), (25, 0.000902286987993942), (26, 0.003312144619340922), (27, 0.00010163405813025485), (28, 1.0886440090901793e-05), (29, 0.000758109828337623), (30, 0.0012249898754869112), (31, 8.885855297719361e-06), (32, 0.0009745604505227973), (33, 0.002999976399983297), (34, 0.00028867306992145706), (35, 0.0069447211376388), (36, 0.0086292

Cada tupla (índice, importancia) indica qué columna se utilizó en algún split del árbol y cuánto aportó a la decisión de dividir los nodos.

Observando las importancias de las variables de nuestro árbol:

- No todas las variables se utilizan en los splits.  
- Solo se usan aquellas que aportan información útil para separar las clases, mientras que las demás quedan con importancia 0 y no aparecen en ningún nodo.  
- Aunque el árbol analiza todas las variables disponibles, solo mantiene en el modelo final las que realmente ayudan a mejorar la clasificación.



**11. ¿Se le ocurre alguna forma de crear un selector de atributos a partir de un árbol de clasificación?**

Aprovechando la importancia de cada variable feature_importances, podemos utilizar el árbol de decisión como selector de atributos.

- Manteniendo únicamente las variables que realmente aportan información al modelo.  
- Las variables con importancia 0 o muy baja pueden eliminarse, ya que no influyen en los splits.  
- Esto permite reducir la dimensionalidad, simplificar el modelo e incluso en algunos casos mejorar la generalización.

En Scikit-Learn se puede hacer fácilmente con SelectFromModel usando el árbol entrenado `decision_tree´.


In [8]:
selector = SelectFromModel(decision_tree, threshold=0.01, prefit=True)
X_selected = selector.transform(X)

print(f"Número de variables originales: {X.shape[1]}")
print(f"Número de variables seleccionadas: {X_selected.shape[1]}")


Número de variables originales: 54
Número de variables seleccionadas: 13


SelectFromModel mira decision_tree.feature_importances_ y compara cada valor con  threshold = 0,01.  
Si la importancia es mayor o igual al threshold, se mantiene; si es menor, se descarta.  
Al hacer X_selected = selector.transform(X) conservamos solo las columnas importantes.



### **2. Regresión mediante árboles de regresión**

#### **2.1. Obtención de los datos**

**Descargue los datos de Sklearn**

In [9]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
X, y = data.data, data.target

**Separe los datos en training (50 %) test (50 %) y realice el preprocesado de los
datos que estime conveniente teniendo en cuenta que el modelo de aprendizaje con
el que va a tratar de resolver el problema es un  ́arbol de regresión. Finalmente
responda a las siguientes preguntas**

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

En el caso del árbol de regresión, no es necesario aplicar técnicas de escalado ni normalización, ya que no son sensibles a la escala de los atributos al realizar sus particiones.

In [11]:
df = pd.DataFrame(X, columns=data.feature_names)
print(df.isnull().sum())      

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
dtype: int64


In [12]:
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(df.describe())


Shape X: (20640, 8)
Shape y: (20640,)
             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486      5.429000      1.096675   1425.476744   
std        1.899822     12.585558      2.474173      0.473911   1132.462122   
min        0.499900      1.000000      0.846154      0.333333      3.000000   
25%        2.563400     18.000000      4.440716      1.006079    787.000000   
50%        3.534800     29.000000      5.229129      1.048780   1166.000000   
75%        4.743250     37.000000      6.052381      1.099526   1725.000000   
max       15.000100     52.000000    141.909091     34.066667  35682.000000   

           AveOccup      Latitude     Longitude  
count  20640.000000  20640.000000  20640.000000  
mean       3.070655     35.631861   -119.569704  
std       10.386050      2.135952      2.003532  
min        0.692308     32.540000   -124.350000  


<span style="color:#2196F3">**PREGUNTAR A LAS NIÑAS QUE HAN HECHO DE PREPROCESADO**</span>


**1. ¿Podría trabajar un arbol de regresión con missing values?**

Sí.

El modelo DecisionTreeRegressor tienen soporte integrado para valores faltantes.
Durante el entrenamiento, el árbol evalúa las divisiones usando los datos disponibles y distribuye los valores faltantes según las reglas aprendidas.

Al predecir, las muestras con valores faltantes se asignan automáticamente al nodo más probable según el comportamiento observado durante el entrenamiento.

Referencia oficial: [https://scikit-learn.org/stable/modules/tree.html#missing-values-support](https://scikit-learn.org/stable/modules/tree.html#missing-values-support)




**2. ¿Es sensible un ́arbol de regresión a las magnitudes de los atributos?**

No.
Los árboles de regresión no son sensibles a la escala ni a las magnitudes de los atributos, ya que las divisiones se basan únicamente en comparaciones (x_j < t) y no en distancias ni relaciones de proporción entre variables.

Por eso no es necesario escalar ni normalizar los datos antes de entrenar el modelo.
Esta es una de las ventajas que destaca la documentación oficial: los árboles de decisión requieren muy poca preparación de datos en comparación con otros modelos como KNN, que sí dependen de la escala.

Referencia oficial: [https://scikit-learn.org/stable/modules/tree.html#](https://scikit-learn.org/stable/modules/tree.html#)


**3. ¿Es sensible un árbol de regresión a la distribución de las etiquetas?**

Sí, en cierta medida.
Los árboles de regresión minimizan el error cuadrático medio (MSE) al crear las divisiones, por lo que los valores de salida muy altos o atípicos (outliers) pueden influir más en el ajuste del modelo.
Esto significa que si las etiquetas (y) están muy desbalanceadas o contienen valores extremos, el árbol puede sobreajustarse a esos casos.

Para evitarlo, la documentación recomienda controlar la complejidad del árbol con parámetros como:

* max_depth → limita la profundidad máxima.
* min_samples_split → exige un mínimo de muestras para dividir un nodo.
* min_samples_leaf → asegura que cada hoja tenga suficientes muestras.

También se puede aplicar una transformación de las etiquetas (por ejemplo, np.log1p(y)) para reducir el impacto de los valores muy grandes.

Referencia oficial: [https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use](https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)



#### **2.2 Implementación de un árbol de regresión**



**1. Construya una clase Nodo**

In [13]:
class Nodo:
    """
    Nodo del árbol de regresión.
    Si es hoja -> value ≠ None y f_index = threshold = None
    Si no es hoja -> define un split con f_index y threshold
    """
    def __init__(self, f_index=None, threshold=None, value=None):
        self.f_index = f_index # Indice del atributo
        self.threshold = threshold # Umbral
        self.value = value # Valor medio (si es hoja)
        self.left = None # Hijo izquierdo
        self.right = None # Hijo derecho

**2. Construya la clase RegressionTree**

In [ ]:
class RegressionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.raiz = None  # Nodo raíz (instancia de Nodo)

    # ----------------------------
    # 1. Cálculo del error regional
    # ----------------------------
    def region_error(self, y):
        """Devuelve la varianza de y o 0 si está vacío."""
        if len(y) > 0:
            return np.var(y)
        else:
            return 0

    # ----------------------------------
    # 2. Búsqueda del mejor atributo y θ
    # ----------------------------------
        
    def _best_split(self, X, y):
        _, n_features = X.shape
        best_feature, best_threshold = None, None
        best_error = float("inf")

        for f in range(n_features):
            values = np.sort(np.unique(X[:, f]))
            if len(values) < 2:
                continue
            # Se decide utilizar el promedio entre valores consecutivos como umbral, en lugar de un valor exacto.
            # Esto permite evaluar los posibles cortes de forma más estable y evita usar valores exactos del conjunto.
            thresholds = (values[:-1] + values[1:]) / 2 
            for t in thresholds:
                left_idx = X[:, f] <= t
                right_idx = ~left_idx
                y_left, y_right = y[left_idx], y[right_idx]
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                n = len(y)
                error = (len(y_left) / n) * self._region_error(y_left) + \
                        (len(y_right) / n) * self._region_error(y_right)
                if error < best_error:
                    best_error = error
                    best_feature = f
                    best_threshold = t

        return best_feature, best_threshold

    # ----------------------------------------
    # 3. Construcción recursiva del árbol
    # ----------------------------------------
    def _construir_nivel(self, X, y, profundidad):
        # Caso base 1: profundidad agotada
        if profundidad == 0:
            return Nodo(value=np.mean(y))

        # Caso base 2: pocas muestras
        if len(y) <= self.min_samples_split:
            return Nodo(value=np.mean(y))

        # Buscar mejor división
        f_opt, t_opt = self._find_best_split(X, y)
        if f_opt is None:
            return Nodo(value=np.mean(y))

        # Crear nodo interno
        nodo = Nodo(f_index=f_opt, threshold=t_opt)

        # Dividir datos
        left_idx = X[:, f_opt] <= t_opt
        right_idx = ~left_idx

        X_left, y_left = X[left_idx], y[left_idx]
        X_right, y_right = X[right_idx], y[right_idx]

        # Hijos
        nodo.left = self.construir_nivel(X_left, y_left, profundidad - 1)
        nodo.right = self.construir_nivel(X_right, y_right, profundidad - 1)

        return nodo

    # ----------------------------
    # 4. Ajustar el modelo
    # ----------------------------
    def fit(self, X, y):
        """Entrena el árbol y define el nodo raíz."""
        self.raiz = self.construir_nivel(np.array(X), np.array(y), self.max_depth)

    # ----------------------------
    # 5. Predicción de una muestra
    # ----------------------------
    def _predict_sample(self, x, nodo):
        if nodo.value is not None:
            return nodo.value

        if x[nodo.f_index] <= nodo.threshold:
            return self._predict_sample(x, nodo.left)
        else:
            return self._predict_sample(x, nodo.right)

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_sample(x, self.raiz) for x in X])

    # ----------------------------
    # 6. Camino de decisión
    # ----------------------------
    def decision_path(self, x):
        """Devuelve un string con las decisiones seguidas hasta la predicción."""
        pasos = []
        nodo = self.raiz
        i = 1

        while nodo.value is None:
            if x[nodo.f_index] <= nodo.threshold:
                pasos.append(f"{i}. Atributo {nodo.f_index} menor o igual que {nodo.threshold}")
                nodo = nodo.left
            else:
                pasos.append(f"{i}. Atributo {nodo.f_index} mayor que {nodo.threshold}")
                nodo = nodo.right
            i += 1

        pasos.append(f"Predicción final = {nodo.value:.3f}")
        return "\n".join(pasos)

#### **2.3. Nos enfrentamos al problema de California Housing**

**1. ¿Cómo evoluciona el error de training y test conforme se aumenta la profundidad del  ́arbol? Base su razonamiento en dos gráficas obtenidas con matplotlib.**

**2. ¿Sobre qué parámetros tendría sentido realizar cross-validation en nuestro modelo?**


**3. Tome un par de muestras y utilizando el método decision_path(x) justifique la predicción dada. Como habrá comprobado, el método decision_path(x) devuelve una lista ordenada de argumentos, suponga que esa lista se desordena: ¿seguiría siendo válida la interpretación de la predicción?**

**4. Para una muestra idéntica a la del apartado anterior, si se entrenara un nuevo árbol, ¿la justificación de su predicción sería la misma?**

**5. Suponga que divide el conjunto de train en dos mitades, entrena dos árboles distintos, uno con cada mitad, y toma una muestra de test. ¿Sería la predicción de la muestra igual en cada árbol? ¿Y la interpretación obtenida con decision_path(x)? ¿Qué repercusión en la interpretación tendría esa situación?**


**6. Utilice la clase DecisionTreeRegressor de Sklearn para resolver de nuevo el problema. Compare el score de ambos modelos y, para alguna muestra, el resultado de decision_path(x) de ambos árboles.**

**7. En base a su implementación ¿cuál es el coste computacional teórico de entrenar un árbol? ¿Y cuál sería el coste en predicción? ¿Y el coste en memoria? Si tuviera que resolver un problema con un conjunto de datos de gran volumen, ¿qué preferiría usar, un árbol o un modelo de regresión lineal?**


### **3. Ensembles**